In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import cv2
import torch
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torchvision
import shutil
import imageio
from IPython import display
from PIL import Image
from glob import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid, save_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
root_dir = "/kaggle/input/butterfly-images40-species/"

In [ ]:
os.makedirs('all_images')

In [ ]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [ ]:
for folder in os.listdir(os.path.join(root_dir, 'train')):
    files = os.listdir(os.path.join(root_dir, 'train', folder))
    for file in files:
        file_name = f'{folder}_{file}'
        shutil.copy2(os.path.join(root_dir, 'train', folder, file), os.path.join('all_images', file_name))

In [ ]:
class Butterfly(Dataset):
    def __init__(self, folder):
        super().__init__()
        self.folder = folder
        self.images = sorted(glob(folder + '/*'))
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, ix):
        image_path = self.images[ix]
        image = Image.open(image_path)
        image = transform(image)
        return image

In [ ]:
data = Butterfly(folder='all_images')

In [ ]:
dataloader = DataLoader(data, batch_size=64, shuffle=True)

# Network Initialization

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(100, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        self.apply(weights_init)
    
    def forward(self, input):
        return self.model(input)

In [ ]:
generator = Generator().to(device)

# Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        self.apply(weights_init)

    def forward(self, input):
        return self.model(input)

In [ ]:
discriminator = Discriminator().to(device)

# Train

In [ ]:
criterion = nn.BCELoss()
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
random_noise = torch.randn(64, 100, 1, 1, device=device)

In [ ]:
d_loss_s = []
g_loss_s = []

In [ ]:
os.makedirs("./butterfly")

In [ ]:
for epoch in range(25):
    N = len(dataloader)
    for i, images in enumerate(dataloader):
        real_data = images.to(device)
        fake_data = generator(torch.randn(len(real_data), 100, 1, 1).to(device)).to(device)
        fake_data = fake_data.detach()

        discriminator_optimizer.zero_grad()
        prediction_real = discriminator(real_data)
        error_real = loss(prediction_real.squeeze(), torch.ones(len(real_data)).to(device))
        error_real.backward()
        prediction_fake = discriminator(fake_data)
        error_fake = loss(prediction_fake.squeeze(), torch.zeros(len(fake_data)).to(device))
        error_fake.backward()
        discriminator_optimizer.step()
        d_loss = error_real + error_fake
            
        fake_data = generator(torch.randn(len(real_data), 100, 1, 1).to(device)).to(device)

        generator_optimizer.zero_grad()
        prediction = discriminator(fake_data)
        g_loss = loss(prediction.squeeze(), torch.ones(len(real_data)).to(device))
        g_loss.backward()
        generator_optimizer.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), d_loss, g_loss))

        d_loss_s.append(d_loss)
        g_loss_s.append(g_loss)

        if i % 95 == 0:
            fake = generator(random_noise)
            save_image(fake.data, f'butterfly/fake_epoch_{epoch:03d}.png', normalize=True)

# Results

In [ ]:
g_loss_s = [g_loss_.cpu().detach().item() for g_loss_ in g_loss_s]
d_loss_s = [d_loss_.cpu().detach().item() for d_loss_ in d_loss_s]

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(g_loss_s)
plt.plot(d_loss_s)
plt.title("Loss Curve")
plt.legend(["Generator Loss", "Discriminator Loss"])
plt.show()

In [ ]:
generator.eval()
sample_images = generator(random_noise).detach().cpu()
grid = make_grid(sample_images, nrow=8, normalize=True)
img = torchvision.transforms.ToPILImage()(grid)
plt.imshow(img)

# GIF

In [ ]:
files = glob("./butterfly/*.png")
gif_file = "./butterfly/dcgan_butterfly.gif"

In [ ]:
with imageio.get_writer(gif_file, mode='I') as writer:
    for file in files:
        image = imageio.imread(file)
        writer.append_data(image)

In [ ]:
display.Image(filename=gif_file)